# PID Controllers - Introduction to Control Systems

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ajthor/lecture-notes/blob/main/docs/pid-controllers.ipynb)

## Learning Objectives

By the end of this lecture, you will be able to:

1. Understand the mathematical foundation of PID controllers
2. Explain the role of proportional (P), integral (I), and derivative (D) components
3. Analyze step responses of different controller configurations
4. Apply basic tuning methods to achieve desired system performance
5. Implement PID controllers in Python using control system libraries

## Introduction

PID (Proportional-Integral-Derivative) controllers are one of the most widely used control algorithms in industry. They provide a systematic approach to controlling dynamic systems by combining three fundamental control actions:

- **Proportional (P)**: Responds to the current error
- **Integral (I)**: Responds to the accumulation of past errors  
- **Derivative (D)**: Responds to the rate of change of error

The PID controller output is given by:

$$u(t) = K_p e(t) + K_i \int_0^t e(\tau) d\tau + K_d \frac{de(t)}{dt}$$

Where:
- $u(t)$ is the controller output
- $e(t)$ is the error signal (setpoint - measured value)
- $K_p$, $K_i$, $K_d$ are the proportional, integral, and derivative gains

In [ ]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import control as ctrl

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Control library version: {ctrl.__version__}")

## Define System Transfer Function

We'll work with a second-order system representing a common plant in control systems. This could represent various physical systems like:
- Mass-spring-damper system
- DC motor
- Chemical process

The transfer function is:

$$G(s) = \frac{1}{s^2 + 2s + 1}$$

This is a critically damped second-order system with natural frequency $\omega_n = 1$ rad/s and damping ratio $\zeta = 1$.

In [ ]:
# Define the plant transfer function G(s) = 1/(s^2 + 2s + 1)
numerator = [1]
denominator = [1, 2, 1]
plant = ctrl.tf(numerator, denominator)

print("Plant Transfer Function:")
print(plant)

# Display pole locations
poles = ctrl.pole(plant)
print(f"\nPole locations: {poles}")

# Check system properties
damping_ratio = 1.0
natural_freq = 1.0
print(f"Natural frequency (ωn): {natural_freq} rad/s")
print(f"Damping ratio (ζ): {damping_ratio}")
print("System type: Critically damped")

## Step Response Analysis - Open Loop

Let's first analyze how our plant responds to a step input without any controller. This gives us the baseline behavior we want to improve with our PID controller.

In [ ]:
# Generate step response for the open-loop plant
t, y = ctrl.step_response(plant, T=10)

plt.figure(figsize=(10, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Open-loop plant response')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (seconds)')
plt.ylabel('Output')
plt.title('Step Response of Open-Loop Plant')
plt.legend()

# Calculate performance metrics
final_value = y[-1]
rise_time_idx = np.where(y >= 0.1 * final_value)[0][0]
rise_time_90_idx = np.where(y >= 0.9 * final_value)[0][0]
rise_time = t[rise_time_90_idx] - t[rise_time_idx]

print(f"Open-loop Performance Metrics:")
print(f"Final Value: {final_value:.3f}")
print(f"Rise Time (10%-90%): {rise_time:.3f} seconds")
print(f"Steady-state Error for unit step: {1 - final_value:.3f}")

plt.tight_layout()
plt.show()

## Implement PID Controller

Now let's implement a PID controller. The PID controller in the s-domain is:

$$C(s) = K_p + \frac{K_i}{s} + K_d s$$

We can also write this as:

$$C(s) = K_p \left(1 + \frac{1}{T_i s} + T_d s\right)$$

Where:
- $T_i = K_p/K_i$ is the integral time constant
- $T_d = K_d/K_p$ is the derivative time constant

In [ ]:
def create_pid_controller(Kp, Ki, Kd):
    """
    Create a PID controller transfer function
    
    Parameters:
    Kp: Proportional gain
    Ki: Integral gain  
    Kd: Derivative gain
    
    Returns:
    PID controller transfer function
    """
    # PID in transfer function form: Kp + Ki/s + Kd*s
    # This becomes: (Kd*s^2 + Kp*s + Ki)/s
    
    if Ki == 0 and Kd == 0:
        # Pure proportional controller
        controller = ctrl.tf([Kp], [1])
    elif Kd == 0:
        # PI controller
        num = [Kp, Ki]
        den = [1, 0]
        controller = ctrl.tf(num, den)
    else:
        # Full PID controller
        num = [Kd, Kp, Ki]
        den = [1, 0]
        controller = ctrl.tf(num, den)
    
    return controller

# Example: Create a PID controller with initial gains
Kp = 2.0  # Proportional gain
Ki = 1.0  # Integral gain
Kd = 0.5  # Derivative gain

pid_controller = create_pid_controller(Kp, Ki, Kd)
print("PID Controller Transfer Function:")
print(pid_controller)
print(f"\nController gains: Kp={Kp}, Ki={Ki}, Kd={Kd}")

## Closed-Loop System Response

Now let's create the closed-loop system by connecting our PID controller to the plant in a feedback configuration:

```
Reference → [+] → PID Controller → Plant → Output
Input       [-]                            ↓
             ↑___________________________|
                    Feedback Loop
```

The closed-loop transfer function is:

$$T(s) = \frac{C(s)G(s)}{1 + C(s)G(s)}$$

In [ ]:
# Create closed-loop system
closed_loop = ctrl.feedback(pid_controller * plant, 1)

print("Closed-Loop Transfer Function:")
print(closed_loop)

# Get step response of closed-loop system
t_cl, y_cl = ctrl.step_response(closed_loop, T=10)

# Plot comparison between open-loop and closed-loop responses
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(t, y, 'b--', linewidth=2, label='Open-loop plant')
plt.plot(t_cl, y_cl, 'r-', linewidth=2, label='Closed-loop with PID')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (seconds)')
plt.ylabel('Output')
plt.title('Step Response Comparison: Open-Loop vs Closed-Loop')
plt.legend()
plt.ylim(-0.1, 1.5)

# Calculate closed-loop performance metrics
final_value_cl = y_cl[-1]
overshoot = (np.max(y_cl) - final_value_cl) / final_value_cl * 100
settling_time_idx = np.where(np.abs(y_cl - final_value_cl) <= 0.02 * final_value_cl)[0]
settling_time = t_cl[settling_time_idx[0]] if len(settling_time_idx) > 0 else t_cl[-1]

print(f"\nClosed-loop Performance Metrics:")
print(f"Final Value: {final_value_cl:.3f}")
print(f"Overshoot: {overshoot:.2f}%")
print(f"Settling Time (2%): {settling_time:.3f} seconds")
print(f"Steady-state Error: {1 - final_value_cl:.6f}")

# Plot error signal
plt.subplot(2, 1, 2)
error = 1 - y_cl  # Error for unit step reference
plt.plot(t_cl, error, 'g-', linewidth=2, label='Error signal')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (seconds)')
plt.ylabel('Error')
plt.title('Error Signal (Reference - Output)')
plt.legend()

plt.tight_layout()
plt.show()

## Compare P, PI, and PID Performance

Let's compare the performance of different controller configurations:
1. **P Controller**: Only proportional action
2. **PI Controller**: Proportional + Integral action  
3. **PID Controller**: Full proportional + Integral + Derivative action

This comparison will help us understand the contribution of each component.

In [ ]:
# Define controller gains for comparison
Kp = 2.0
Ki = 1.0
Kd = 0.5

# Create different controller types
p_controller = create_pid_controller(Kp, 0, 0)      # P only
pi_controller = create_pid_controller(Kp, Ki, 0)    # PI
pid_controller = create_pid_controller(Kp, Ki, Kd)  # PID

# Create closed-loop systems
cl_p = ctrl.feedback(p_controller * plant, 1)
cl_pi = ctrl.feedback(pi_controller * plant, 1)
cl_pid = ctrl.feedback(pid_controller * plant, 1)

# Get step responses
t_sim = np.linspace(0, 10, 1000)
t_p, y_p = ctrl.step_response(cl_p, T=t_sim)
t_pi, y_pi = ctrl.step_response(cl_pi, T=t_sim)
t_pid, y_pid = ctrl.step_response(cl_pid, T=t_sim)

# Plot comparison
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.plot(t_p, y_p, 'b-', linewidth=2, label='P Controller')
plt.plot(t_pi, y_pi, 'g-', linewidth=2, label='PI Controller')
plt.plot(t_pid, y_pid, 'r-', linewidth=2, label='PID Controller')
plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (seconds)')
plt.ylabel('Output')
plt.title('Step Response Comparison: P vs PI vs PID Controllers')
plt.legend()
plt.ylim(-0.1, 1.8)

# Plot error signals
plt.subplot(3, 1, 2)
error_p = 1 - y_p
error_pi = 1 - y_pi  
error_pid = 1 - y_pid
plt.plot(t_p, error_p, 'b-', linewidth=2, label='P Controller Error')
plt.plot(t_pi, error_pi, 'g-', linewidth=2, label='PI Controller Error')
plt.plot(t_pid, error_pid, 'r-', linewidth=2, label='PID Controller Error')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (seconds)')
plt.ylabel('Error')
plt.title('Error Signals Comparison')
plt.legend()

# Performance metrics comparison
plt.subplot(3, 1, 3)
controllers = ['P', 'PI', 'PID']
steady_state_errors = [
    abs(1 - y_p[-1]),
    abs(1 - y_pi[-1]), 
    abs(1 - y_pid[-1])
]
overshoots = [
    (np.max(y_p) - y_p[-1]) / y_p[-1] * 100 if y_p[-1] > 0 else 0,
    (np.max(y_pi) - y_pi[-1]) / y_pi[-1] * 100 if y_pi[-1] > 0 else 0,
    (np.max(y_pid) - y_pid[-1]) / y_pid[-1] * 100 if y_pid[-1] > 0 else 0
]

x_pos = np.arange(len(controllers))
width = 0.35

plt.bar(x_pos - width/2, steady_state_errors, width, label='Steady-State Error', alpha=0.8)
plt.bar(x_pos + width/2, np.array(overshoots)/100, width, label='Overshoot (normalized)', alpha=0.8)

plt.xlabel('Controller Type')
plt.ylabel('Performance Metric')
plt.title('Performance Metrics Comparison')
plt.xticks(x_pos, controllers)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print numerical comparison
print("Performance Comparison:")
print("-" * 50)
print(f"{'Controller':<10} {'SS Error':<12} {'Overshoot %':<12} {'Final Value':<12}")
print("-" * 50)
print(f"{'P':<10} {steady_state_errors[0]:<12.4f} {overshoots[0]:<12.2f} {y_p[-1]:<12.4f}")
print(f"{'PI':<10} {steady_state_errors[1]:<12.4f} {overshoots[1]:<12.2f} {y_pi[-1]:<12.4f}")
print(f"{'PID':<10} {steady_state_errors[2]:<12.4f} {overshoots[2]:<12.2f} {y_pid[-1]:<12.4f}")

## Interactive PID Tuning

Let's create an interactive example where you can experiment with different PID gains and see their effects in real-time. This will help you understand how each parameter affects system performance.

In [ ]:
def plot_pid_response(Kp_val, Ki_val, Kd_val):
    """
    Plot step response for given PID gains
    """
    # Create PID controller with given gains
    controller = create_pid_controller(Kp_val, Ki_val, Kd_val)
    
    # Create closed-loop system
    try:
        closed_loop = ctrl.feedback(controller * plant, 1)
        t, y = ctrl.step_response(closed_loop, T=10)
        
        plt.figure(figsize=(12, 8))
        
        # Step response plot
        plt.subplot(2, 1, 1)
        plt.plot(t, y, 'b-', linewidth=2, label=f'PID Response (Kp={Kp_val}, Ki={Ki_val}, Kd={Kd_val})')
        plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
        plt.grid(True, alpha=0.3)
        plt.xlabel('Time (seconds)')
        plt.ylabel('Output')
        plt.title('Step Response with PID Controller')
        plt.legend()
        plt.ylim(-0.5, 2.0)
        
        # Error plot
        plt.subplot(2, 1, 2)
        error = 1 - y
        plt.plot(t, error, 'r-', linewidth=2, label='Error Signal')
        plt.grid(True, alpha=0.3)
        plt.xlabel('Time (seconds)')
        plt.ylabel('Error')
        plt.title('Error Signal')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Calculate and display performance metrics
        final_value = y[-1]
        overshoot = (np.max(y) - final_value) / final_value * 100 if final_value > 0 else 0
        steady_state_error = abs(1 - final_value)
        
        print(f"Performance Metrics for Kp={Kp_val}, Ki={Ki_val}, Kd={Kd_val}:")
        print(f"  Final Value: {final_value:.4f}")
        print(f"  Overshoot: {overshoot:.2f}%") 
        print(f"  Steady-State Error: {steady_state_error:.6f}")
        
        # Check stability
        poles = ctrl.pole(closed_loop)
        stable = all(np.real(pole) < 0 for pole in poles)
        print(f"  System Stable: {stable}")
        
    except Exception as e:
        print(f"Error creating system with Kp={Kp_val}, Ki={Ki_val}, Kd={Kd_val}: {e}")

# Try different PID tuning examples
print("Example 1: Well-tuned PID")
plot_pid_response(2.0, 1.0, 0.5)

print("\n" + "="*60)
print("Example 2: High Proportional Gain (may cause oscillations)")
plot_pid_response(10.0, 1.0, 0.5)

print("\n" + "="*60)
print("Example 3: High Integral Gain (may cause overshoot)")  
plot_pid_response(2.0, 5.0, 0.5)

## Tuning Guidelines and Best Practices

### Manual Tuning Method

A simple approach to manual PID tuning:

1. **Start with all gains at zero**
2. **Increase Kp** until you get a reasonable rise time, but expect some overshoot
3. **Add Ki** to eliminate steady-state error, but watch for increased overshoot
4. **Add Kd** to reduce overshoot and improve stability

### Effects of Each Gain:

| Parameter | Rise Time | Overshoot | Settling Time | SS Error | Stability |
|-----------|-----------|-----------|---------------|----------|-----------|
| **Kp ↑**  | Decrease  | Increase  | Small change  | Decrease | Degrade   |
| **Ki ↑**  | Decrease  | Increase  | Increase      | Eliminate| Degrade   |
| **Kd ↑**  | Small change | Decrease | Decrease    | No effect| Improve   |

### Common Problems:

- **Too much overshoot**: Reduce Kp or Ki, or increase Kd
- **Slow response**: Increase Kp
- **Steady-state error**: Add or increase Ki
- **Oscillations**: Reduce Kp and Ki, increase Kd
- **Instability**: Reduce all gains, especially Kp and Ki

## Summary and Exercises

### Key Takeaways:

1. **PID controllers** combine three control actions to achieve desired performance
2. **Proportional (P)** action provides fast response but may have steady-state error
3. **Integral (I)** action eliminates steady-state error but can cause overshoot
4. **Derivative (D)** action improves stability and reduces overshoot
5. **Tuning** is often iterative and requires balancing competing objectives

### Try This Yourself:

Experiment with the `plot_pid_response()` function using different gain values:

```python
# Try these different tuning scenarios:
# 1. P-only controller: plot_pid_response(Kp, 0, 0)  
# 2. PI controller: plot_pid_response(Kp, Ki, 0)
# 3. PD controller: plot_pid_response(Kp, 0, Kd)
# 4. Full PID: plot_pid_response(Kp, Ki, Kd)
```

### Next Steps:

- Learn about **Ziegler-Nichols tuning method**
- Study **root locus techniques** for controller design
- Explore **frequency domain analysis** (Bode plots)
- Investigate **modern control methods** (LQR, MPC)

---

**References:**
- Åström, K. J., & Hägglund, T. (2006). *Advanced PID control*. ISA-The Instrumentation, Systems, and Automation Society.
- Franklin, G. F., Powell, J. D., & Emami-Naeini, A. (2014). *Feedback control of dynamic systems*. Pearson.